In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, TensorDataset
import pandas as pd
from sklearn.metrics import mean_squared_error
import numpy as np
from sentence_transformers import SentenceTransformer

In [2]:

import random

torch.manual_seed(42)
np.random.seed(42)
random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)
    torch.cuda.manual_seed_all(42)

train_df = pd.read_csv("train.csv")
val_df = pd.read_csv("val.csv")
test_df = pd.read_csv("test.csv")

In [3]:
model = SentenceTransformer("all-MiniLM-L6-v2")
X_train = model.encode(train_df["query"].tolist(), convert_to_tensor=True)
X_val = model.encode(val_df["query"].tolist(), convert_to_tensor=True)
X_test = model.encode(test_df["query"].tolist(), convert_to_tensor=True)

X_train = X_train.unsqueeze(1)
X_val   = X_val.unsqueeze(1)
X_test  = X_test.unsqueeze(1)

y_train = torch.tensor(train_df["carb"].values, dtype=torch.float32).unsqueeze(1)
y_val   = torch.tensor(val_df["carb"].values, dtype=torch.float32).unsqueeze(1)
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val, y_val), batch_size=32)

X_train.shape

torch.Size([8000, 1, 384])

In [4]:
class RMSELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, yhat, y):
        return torch.sqrt(self.mse(yhat, y))

In [56]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size=384, hidden_size=128, num_layers=3, dropout=0.4, bidirectional=False):
        super(RNN, self).__init__()
        self.rnn = nn.RNN(
            input_size=input_size, 
            hidden_size=hidden_size, 
            num_layers=num_layers, 
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0, 
            bidirectional=bidirectional, 
            nonlinearity='relu'
            )
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        return self.fc(out[:, -1, :])    


In [57]:
model = RNN()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-5)
criterion = nn.MSELoss()

epochs = 100

for epoch in range(epochs):
    model.train()
    total_loss = 0
    for x_batch, y_batch in train_loader:
        optimizer.zero_grad()
        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    avg_loss = total_loss / len(train_loader)
    rmse = avg_loss ** 0.5
    print(f"Epoch {epoch+1}, Train MSE: {avg_loss:.4f}, RMSE: {rmse:.2f} grams")

    

Epoch 1, Train MSE: 1673.9194, RMSE: 40.91 grams
Epoch 2, Train MSE: 1368.5001, RMSE: 36.99 grams
Epoch 3, Train MSE: 1228.1904, RMSE: 35.05 grams
Epoch 4, Train MSE: 1122.6308, RMSE: 33.51 grams
Epoch 5, Train MSE: 1006.0295, RMSE: 31.72 grams
Epoch 6, Train MSE: 928.0297, RMSE: 30.46 grams
Epoch 7, Train MSE: 791.1488, RMSE: 28.13 grams
Epoch 8, Train MSE: 798.6103, RMSE: 28.26 grams
Epoch 9, Train MSE: 674.4348, RMSE: 25.97 grams
Epoch 10, Train MSE: 649.3596, RMSE: 25.48 grams
Epoch 11, Train MSE: 554.6990, RMSE: 23.55 grams
Epoch 12, Train MSE: 570.3453, RMSE: 23.88 grams
Epoch 13, Train MSE: 571.8366, RMSE: 23.91 grams
Epoch 14, Train MSE: 542.9341, RMSE: 23.30 grams
Epoch 15, Train MSE: 484.0213, RMSE: 22.00 grams
Epoch 16, Train MSE: 481.9888, RMSE: 21.95 grams
Epoch 17, Train MSE: 449.5115, RMSE: 21.20 grams
Epoch 18, Train MSE: 445.0863, RMSE: 21.10 grams
Epoch 19, Train MSE: 415.8509, RMSE: 20.39 grams
Epoch 20, Train MSE: 412.0339, RMSE: 20.30 grams
Epoch 21, Train MSE: 374

In [58]:
model.eval()
total_val_loss = 0
all_preds = []
all_targets = []

with torch.no_grad():
    for x_batch, y_batch in val_loader:

        preds = model(x_batch)
        loss = criterion(preds, y_batch)
        total_val_loss += loss.item()

        all_preds.append(preds.cpu())
        all_targets.append(y_batch.cpu())

avg_val_loss = total_val_loss / len(val_loader)
rmse = avg_val_loss ** 0.5
print(f"Validation MSE: {avg_val_loss:.4f}, RMSE: {rmse:.2f} grams")

Validation MSE: 345.0746, RMSE: 18.58 grams


In [59]:
model.eval()
with torch.no_grad():
    preds = model(X_val).squeeze().numpy()

# Add prediction column and save
test_df["carb"] = preds
test_df.to_csv("test_with_predictions_transformer_rnn.csv", index=False)